In [ ]:
import pandas as pd

In [ ]:
import networkx as nx

edgotype = nx.read_gexf("data/y2hEdgotyping/edgotype.gefx")

In [ ]:
import os
import pandas as pd
seqFiles = [pd.read_csv(f"data/y2hEdgotyping/uniprotScan/sequence_{i}.tsv",delimiter="\t") for i in range(6)]

uniprotMatches = pd.concat(seqFiles)
def mergeWithUniprot(graph):
    for node in graph.nodes(data=True):
        seq = node[1]["seq"]
        up = uniprotMatches[(uniprotMatches.Sequence == seq) & \
                            (uniprotMatches.Reviewed == "reviewed") & \
                           (uniprotMatches.Organism == "Homo sapiens (Human)")]
        graph.nodes[node[0]]["uniprotMatches"] = up
        alphafoldStructures = []
        for uniprot_id in graph.nodes[node[0]]["uniprotMatches"]["Entry"]:
            fp = f"/data/dzeiberg/alphafold/predictions/AF-{uniprot_id}-F1-model_v4.pdb.gz"
            if os.path.isfile(fp):
                alphafoldStructures.append(fp)
        graph.nodes[node[0]]["alphafoldStructures"] = alphafoldStructures
    return graph

In [ ]:
edgotype_m = mergeWithUniprot(edgotype)

# Analyze Hit Rate of Edgotype Nodes with Uniprot

In [ ]:
import numpy as np

## Train Uniprot Matches

In [ ]:
np.unique([n[1]["uniprotMatches"].shape[0] for n in edgotype_m.nodes(data=True)],
          return_counts=True)

## Train Alphafold Matches

In [ ]:
np.unique([len(n[1]["alphafoldStructures"]) for n in edgotype_m.nodes(data=True)],
          return_counts=True)

In [ ]:
next(iter(edgotype_m.edges(data=True)))

In [ ]:
next(iter(edgotype_m.nodes(data=True)))[1]["uniprotMatches"]

In [ ]:
nx.write_gexf(edgotype_m,"data/y2hEdgotyping/edgotype_x_uniprot.gefx")